In [2]:
# Dash/Jupyter
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

# Dash components
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl

# Data/Viz
import pandas as pd
import numpy as np
import plotly.express as px

import base64, os

# CRUD module
from animal_shelter import AnimalShelter

# -------------------------------
# Model / DB Connection
# -------------------------------
db = AnimalShelter("aacuser", "SNHU1234")   


df = pd.DataFrame.from_records(db.read({}, {"_id": False}))
df = df.drop(columns=["_id"], errors="ignore")

# -------------------------------
# App / Layout
# -------------------------------
app = JupyterDash(__name__)

# Logo
logo_path = "Grazioso Salvare Logo.png" 
logo_b64 = base64.b64encode(open(logo_path, "rb").read()).decode("utf-8") if os.path.exists(logo_path) else None
UNIQUE_ID = "Amanda Celi, CS-340" 

app.layout = html.Div([

    html.Div([
        html.Center(html.B(html.H1("CS-340 Dashboard"))),
        html.Center(html.Div(UNIQUE_ID, style={"fontWeight": "bold", "marginTop": "4px"})),
        html.Center(html.Img(src=f"data:image/png;base64,{logo_b64}", style={"height": "48px"})) if logo_b64 else html.Span(),
    ], style={"marginBottom": "8px"}),
    html.Hr(),


    html.Div([
        html.Label("Interactive Filter Options"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": " Water Rescue", "value": "Water"},
                {"label": " Mountain or Wilderness Rescue", "value": "Mountain"},
                {"label": " Disaster or Individual Tracking", "value": "Disaster"},
                {"label": " Reset", "value": "Reset"}
            ],
            value="Reset",
            inline=True
        ),
        html.Div(id="records-count", style={"marginTop": "6px", "fontStyle": "italic"})
    ], style={"marginBottom": "10px"}),
    html.Hr(),


    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns],
        data=df.to_dict("records"),
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px", "fontFamily": "Arial", "fontSize": 12},
        style_header={"backgroundColor": "#f2f2f2", "fontWeight": "bold"},
    ),

    html.Br(),
    html.Hr(),


    html.Div(className="row", style={"display": "flex", "gap": "16px"}, children=[
        html.Div(id="graph-id", className="col s12 m6", style={"flex": "1 1 50%"}),
        html.Div(id="map-id",   className="col s12 m6", style={"flex": "1 1 50%"}),
    ])
])

# -------------------------------
# Controller / Callbacks
# -------------------------------

@app.callback(
    Output("datatable-id", "data"),
    Output("records-count", "children"),
    Input("filter-type", "value")
)
def update_table(filter_value):
    # Queries per spec
    WATER_QUERY = {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    }
    MOUNTAIN_QUERY = {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    }
    DISASTER_QUERY = {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }

    if filter_value == "Water":
        query = WATER_QUERY
    elif filter_value == "Mountain":
        query = MOUNTAIN_QUERY
    elif filter_value == "Disaster":
        query = DISASTER_QUERY
    else:
        query = {}

    recs = db.read(query, {"_id": False})
    dff = pd.DataFrame.from_records(recs).drop(columns=["_id"], errors="ignore")
    return dff.to_dict("records"), f"{len(dff)} record(s) shown"


@app.callback(
    Output("graph-id", "children"),
    Input("datatable-id", "data")
)
def update_chart(table_data):
    try:
        if not table_data:
            return html.Div("No data to chart.", style={"padding": "12px", "fontStyle": "italic"})
        dff = pd.DataFrame(table_data)
        if dff.empty:
            return html.Div("No data to chart.", style={"padding": "12px", "fontStyle": "italic"})
        if "breed" not in dff.columns:
            return html.Div("No 'breed' field in data.", style={"padding": "12px", "fontStyle": "italic"})

        vc = dff["breed"].fillna("Unknown").value_counts()
        if vc.empty:
            return html.Div("No chartable values for 'breed'.", style={"padding": "12px", "fontStyle": "italic"})
        if len(vc) > 8:
            top = vc.head(8)
            other = pd.Series({"Other": vc.iloc[8:].sum()})
            vc = pd.concat([top, other])

        fig = px.pie(values=vc.values, names=vc.index, title="Breed Distribution (Filtered)")
        fig.update_traces(textposition="inside", textinfo="percent+label")
        return dcc.Graph(figure=fig)
    except Exception as e:
        return html.Pre(f"Chart error: {e}", style={"color": "crimson", "whiteSpace": "pre-wrap"})


@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "data"),
    Input("datatable-id", "selected_rows")
)
def update_map(table_data, selected_rows):
    def base_map(center=[30.75, -97.48], zoom=10):
        return dl.Map(style={"width": "100%", "height": "500px"}, center=center, zoom=zoom, children=[dl.TileLayer(id="base-layer-id")])

    if not table_data:
        return base_map()
    dff = pd.DataFrame(table_data)
    if dff.empty:
        return base_map()

    row_idx = selected_rows[0] if selected_rows else 0
    row_idx = min(max(row_idx, 0), len(dff) - 1)
    record = dff.iloc[row_idx].to_dict()


    def find_col(cands):
        lower = {c.lower(): c for c in dff.columns}
        for c in cands:
            if c in lower:
                return lower[c]
        return None

    lat_col = find_col(["location_lat", "lat", "latitude"])
    lon_col = find_col(["location_long", "lon", "lng", "longitude"])
    if lat_col is None or lon_col is None:
        return base_map()

    try:
        lat = float(record.get(lat_col))
        lon = float(record.get(lon_col))
    except (TypeError, ValueError):
        return base_map()

    name  = str(record.get("name", "Animal"))
    breed = str(record.get("breed", ""))

    return [
        dl.Map(style={"width": "100%", "height": "500px"}, center=[lat, lon], zoom=12, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed if breed else name),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ])
    ]

app.run_server()


Dash app running on https://presstruck-veteranextreme-3000.codio.io/proxy/8050/
